In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [2]:
target_file_dir = "../../datas/"
target_file_names = [
    "RealEstateData_20111_20164_13_main",
    "RealEstateData_20171_20234_13_main",
    "RealEstateData_20111_20234_14_main"
]
target_file_end = ".csv"
df_origin = pd.DataFrame()
for file_name in target_file_names:
    df = pd.read_csv(target_file_dir + file_name + target_file_end)
    df_origin = pd.concat([df_origin, df], axis=0)

In [74]:
df_origin.query('Type==@target_type').shape

(254632, 16)

In [82]:
type_no = 1
target_col = 'Structure'

type_list = ["宅地(土地と建物)", '中古マンション等']
target_type = type_list[type_no]

df_origin.query('Type==@target_type').pivot_table(index=target_col, values='Area', aggfunc='count').sort_values('Area').tail(30)

,Area
Structure,
ＲＣ、軽量鉄骨造,1
ＳＲＣ、ＲＣ、鉄骨造,1
ＳＲＣ、鉄骨造,7
木造,9
軽量鉄骨造,19
ＲＣ、鉄骨造,77
ＳＲＣ、ＲＣ,162
鉄骨造,1744
ＳＲＣ,61369


In [ ]:
df_tmp.pivot_table

In [51]:
# パラメータ
TradePriceThrethold = 15000
AgeAtTradeThrethold = 50

# マンション
use_cols_non_cate = ["TradePrice","Area","TradeYear","AgeAtTrade", 'TotalFloorArea', 'CoverageRatio', "FloorAreaRatio"] #Municipality Prefecture
category_cols = ['Municipality', 'Structure' ,'Type']

#一軒家
# use_cols_non_cate = ["TradePrice","Area","TradeYear","AgeAtTrade", "CoverageRatio"] #Municipality Prefecture
# category_cols = ['Municipality', 'Structure', 'TradeQuarter']

use_cols = use_cols_non_cate + category_cols
AreaThrethold = 200 # if select_type == "宅地(土地と建物)" else 150 if select_type == "中古マンション等" else 0

# データ読み込み
col_choice_extract = (~df_origin.AgeAtTrade.isnull())
col_choice_threthold = (df_origin.TradePrice<TradePriceThrethold)&(df_origin.Area<AreaThrethold)&(df_origin.AgeAtTrade<AgeAtTradeThrethold)
df_base = df_origin.loc[col_choice_extract&col_choice_threthold,use_cols].copy()
print(df_origin.shape[0], df_origin.loc[col_choice_extract,:].shape[0],df_base.shape[0])

451808 433600 402491


In [52]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

df = df_base.copy()

#Municipalityを数字に変換
mapping_dict = {}
for category_col in category_cols:
    df[category_col], mapping_dict[category_col] = pd.factorize(df[category_col]) 

# 目的変数と説明変数に分割
y = df['TradePrice']
X = df.drop('TradePrice', axis=1)

# パラメータの設定
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 63,
    'learning_rate': 0.05,
    'bagging_freq': 1, 
    'feature_fraction': 0.9
}

# 訓練データとテストデータに分割
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
verbose_eval = 1

# テストデータと学習データ分割
# 参考：https://qiita.com/c60evaporator/items/2b7a2820d575e212bcf4
X_train_raw, X_test, y_train_raw, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
# early_stopping用の評価データをさらに分割
X_train, X_valid, y_train, y_valid = train_test_split(X_train_raw, y_train_raw, test_size=0.2, random_state=42)

###### ここからがLightGBMの実装 ######
# データをDatasetクラスに格納
dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=category_cols)  # 学習用
dvalid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=category_cols)  # early_stopping用

# LightGBMモデルの訓練
model = lgb.train(params, dtrain, 
                  num_boost_round=10000, 
                  valid_sets=[dvalid], 
                  callbacks=[lgb.early_stopping(stopping_rounds=20, 
                                verbose=True), # early_stopping用コールバック関数
                           lgb.log_evaluation(verbose_eval)])

# モデルの評価（MAE）
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
mae = mean_absolute_error(y_test, y_pred)

print('Mean Absolute Error:', mae)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007859 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 329
[LightGBM] [Info] Number of data points in the train set: 305892, number of used features: 9
[LightGBM] [Info] Start training from score 3524.117339
[1]	valid_0's l1: 1505.31
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l1: 1451.95
[3]	valid_0's l1: 1401.86
[4]	valid_0's l1: 1353.85
[5]	valid_0's l1: 1308.53
[6]	valid_0's l1: 1265.87
[7]	valid_0's l1: 1225.81
[8]	valid_0's l1: 1188.68
[9]	valid_0's l1: 1153.6
[10]	valid_0's l1: 1120.69
[11]	valid_0's l1: 1089.54
[12]	valid_0's l1: 1059.72
[13]	valid_0's l1: 1031.97
[14]	valid_0's l1: 1006.74
[15]	valid_0's l1: 982.535
[16]	valid_0's l1: 959.147
[17]	valid_0's l1: 937.675
[18]	valid_0's l1: 917.213
[19]	valid_0's l1: 903.396
[20]	valid_0's l1: 885.243
[

In [53]:
# モデルの評価（MAE） FloorAreaRatio
X_test_1 = X_test.query('Type==0')
X_test_m = X_test.query('Type==1')
y_test_1 = y_test[(X_test.Type==0)]
y_test_m = y_test[X_test.Type==1]

for X_test_, y_test_ in [[X_test_1, y_test_1], [X_test_m, y_test_m]]:
    y_pred = model.predict(X_test_, num_iteration=model.best_iteration)
    mae = mean_absolute_error(y_test_, y_pred)
    print(mae) #575

642.723719058127
472.9183625491783


In [13]:
mapping_dict['Type']

Index(['宅地(土地と建物)', '中古マンション等'], dtype='object')

In [234]:
df = df_origin.copy()
for col in ['TotalFloorArea', 'CoverageRatio', 'FloorAreaRatio']:
    df['isnull_'] = df_origin[col].isnull()
    print(col)
    display(df.pivot_table(index='isnull_', columns='Type',values='Area', aggfunc='count'))

TotalFloorArea


Type,中古マンション等,宅地(土地と建物)
isnull_,,
False,NaN,188336.0
True,254632.0,8840.0


CoverageRatio


Type,中古マンション等,宅地(土地と建物)
isnull_,,
False,250390,195380
True,4242,1796


FloorAreaRatio


Type,中古マンション等,宅地(土地と建物)
isnull_,,
False,250390,195380
True,4242,1796


,Area
Municipality,
あきる野市,21
愛甲郡愛川町,29
中郡二宮町,32
武蔵村山市,54
足柄上郡開成町,83
...,...
新宿区,8804
港区,9124
世田谷区,9949


In [106]:
# マンション用
from itertools import product

Type_list = ['宅地(土地と建物)']
Area_list = [x for x in range(50,101,10)] + [150,200]
TradeYear_list = [2010,2020,2025]
AgeAtTrade_list = [0,10,20, 30, 40, 50]
Municipality_list = [x for x in df_origin.Municipality.unique()]
Structure_list = ['木造']
TotalFloorArea_list = [x for x in range(50,201,50)] + [np.nan]
CoverageRatio_list = [40,50,60,80]
FloorAreaRatio_list = [100,200,300]
#['1K', '1LDK', '1R', '2LDK', '3LDK', '4LDK']

Tyoe_list_2 = ['中古マンション等']
Municipality_for_m = df_origin.query('Type=="中古マンション等"').pivot_table(index='Municipality', values='Area', aggfunc='count').sort_values('Area').query('Area>20').index
Municipality_list_2 = [x for x in Municipality_for_m]
TotalFloorArea_list_2 = [np.nan]
Area_list_2 = [x for x in range(20,101,10)] + [150]
FloorAreaRatio_list_2 = [200,300,400,500,600]
CoverageRatio_list_2 = [60,80]
Structure_list_2 = ['ＲＣ']


# TotalFloorArea_list = [0,999]
# CoverageRatio_list = [0,999]
# FloorAreaRatio_list = [0,999]
# CoverageRatio_list_2 = [0,999]
# FloorAreaRatio_list_2 = [0,999]

# 各リストの組み合わせを生成
combinations= list(product(Area_list, TradeYear_list, AgeAtTrade_list, Municipality_list, Structure_list, Type_list,
                            TotalFloorArea_list, CoverageRatio_list, FloorAreaRatio_list))

combinations_2= list(product(Area_list_2, TradeYear_list, AgeAtTrade_list, Municipality_list_2, Structure_list_2, Tyoe_list_2,
                            TotalFloorArea_list_2, CoverageRatio_list_2, FloorAreaRatio_list_2))

# リストからDataFrameを作成
df = pd.DataFrame(combinations, columns=['Area', 'TradeYear', 'AgeAtTrade', 'Municipality', 'Structure', 'Type', 
                                         'TotalFloorArea','CoverageRatio', 'FloorAreaRatio'])
df2 = pd.DataFrame(combinations_2, columns=['Area', 'TradeYear', 'AgeAtTrade', 'Municipality', 'Structure', 'Type', 
                                         'TotalFloorArea','CoverageRatio', 'FloorAreaRatio'])
df_dummy_m = pd.concat([df,df2])

# 結果を表示
print(df_dummy_m.shape)

# カテゴリー変数を、モデル作成時の変換マップの値で置換
for category_col in category_cols:
    df_dummy_m[category_col] = df_dummy_m[category_col].map({val: i for i, val in enumerate(mapping_dict[category_col])})

# model.predict(df_dummy_m.tail(50))


(1187280, 9)


In [107]:
df_origin.shape

(451808, 16)

In [87]:
mapping_dict['Type']

Index(['宅地(土地と建物)', '中古マンション等'], dtype='object')

In [90]:
df_dumm_2 = df_dummy_m.query('Type==0 and TradeYear==2020 and  AgeAtTrade<30')
df_dumm_2['TradePrice'] = model.predict(df_dumm_2)
df_dumm_2.shape

<ipython-input-90-88ede40913b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dumm_2['TradePrice'] = model.predict(df_dumm_2)


(74880, 10)

In [93]:
df_dumm_2.pivot_table(index=['AgeAtTrade','FloorAreaRatio'], values='TradePrice', aggfunc='mean').tail(50)

TradePrice
AgeAtTrade FloorAreaRatio             
0          80              6313.130479
           100             6313.130479
           150             6313.130479
           200             6313.130479
           300             6313.130479
5          80              5879.787940
           100             5879.787940
           150             5879.787940
           200             5879.787940
           300             5879.787940
10         80              5343.463109
           100             5343.463109
           150             5343.463109
           200             5343.463109
           300             5343.463109
20         80              4492.523200
           100             4492.523200
           150             4492.523200
           200             4492.523200
           300             4492.523200

In [60]:
df_dumm_2.pivot_table(index=['CoverageRatio', 'TotalFloorArea', 'FloorAreaRatio'], values='TradePrice', aggfunc='mean').tail(50)

TradePrice
CoverageRatio TotalFloorArea FloorAreaRatio             
0             0              0               6359.579753
                             999             6359.579753
              999            0               1082.534128
                             999             1082.534128
999           0              0               6188.432404
                             999             6188.432404
              999            0                991.610218
                             999              991.610218

In [42]:
df_dummy_m.tail(40)

,Area,TradeYear,AgeAtTrade,Municipality,Structure,Type,TotalFloorArea,CoverageRatio,FloorAreaRatio
1965560,120,2030,50,113.0,0,1,NaN,60,150
1965561,120,2030,50,113.0,0,1,NaN,60,200
1965562,120,2030,50,113.0,0,1,NaN,60,300
1965563,120,2030,50,113.0,0,1,NaN,60,400
1965564,120,2030,50,113.0,0,1,NaN,60,500
1965565,120,2030,50,113.0,0,1,NaN,80,80
1965566,120,2030,50,113.0,0,1,NaN,80,100
1965567,120,2030,50,113.0,0,1,NaN,80,150
1965568,120,2030,50,113.0,0,1,NaN,80,200
1965569,120,2030,50,113.0,0,1,NaN,80,300


In [19]:
df_origin.shape

(451808, 16)

In [206]:
df.query('Area==80 and Type=="宅地(土地と建物)"').head(30)

,Area,TradeYear,AgeAtTrade,Municipality,Structure,Type,TradePrice_Predict
0,80,2016,0,千代田区,鉄骨造,宅地(土地と建物),12864.728048
2,80,2016,0,千代田区,軽量鉄骨造,宅地(土地と建物),12385.201625
4,80,2016,0,千代田区,ＳＲＣ,宅地(土地と建物),11797.001504
6,80,2016,0,千代田区,木造,宅地(土地と建物),10521.365756
8,80,2016,0,千代田区,ＲＣ,宅地(土地と建物),13691.705956
10,80,2016,0,中央区,鉄骨造,宅地(土地と建物),12048.626119
12,80,2016,0,中央区,軽量鉄骨造,宅地(土地と建物),10520.998341
14,80,2016,0,中央区,ＳＲＣ,宅地(土地と建物),9373.514646
16,80,2016,0,中央区,木造,宅地(土地と建物),8663.026633
18,80,2016,0,中央区,ＲＣ,宅地(土地と建物),11060.681972


In [166]:
df_base.query("Area<80 and Area>60 and  AgeAtTrade<10 and Municipality=='横浜市中区'").pivot_table(index='FloorPlan', values='TradePrice', aggfunc=['mean', 'count'])

,mean,count
,TradePrice,TradePrice
FloorPlan,,
1K,2843.333333,3
1LDK,4017.647059,17
2LDK,4804.651163,43
3LDK,4386.193548,124
4LDK,3800.000000,1
